In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd

import sys
sys.path.append("../")
from bargains_linear import * 

In [10]:
def solve_lamb(df):
    p1 =  df['p1'].mean()
    phi1 = df['phi1'].mean()
    phi2 = df['phi2'].mean()
    s1 = df['s1'].mean()
    s2 = df['s2'].mean()
    lambda_val = ( df['p1'] - (2*df['phi1'] + df['phi2'])/3 ).mean()
    
    return lambda_val

def solve_pi_seq_helper(p2,phi1,phi2,s2,lambda_val):
    
    # Calculate the first equation
    pi_2 = (p2  - phi2 ) * s2
    
    # Calculate the se2*cond equation
    pi_h = s2 * (phi2 - phi1) 

    # Calculate the derivative of the first equation with respect to phi1t
    dpi_2_dphi2 = -1*(3 * lambda_val - phi2 + phi1) / (9 * lambda_val)
    
    # Calculate the derivative of the second equation with respect to phi1t
    dpi_h_dphi2 = -1*(phi2-phi1)/(6 * lambda_val) + s2
    

    return (dpi_h_dphi2*pi_2)/( (dpi_h_dphi2*pi_2 ) - (dpi_2_dphi2*pi_h)  )


def solve_beta(df):
    p1 =  df['p1'].mean()
    p2 =  df['p2'].mean()
    phi1 = df['phi1'].mean()
    phi2 = df['phi2'].mean()
    s1 = df['s1'].mean()
    s2 = df['s2'].mean()
    
    lambda_val = solve_lamb(df)
    #print(p2,phi1,phi2,s2,lambda_val)
    
    beta2 = solve_pi_seq_helper(p2,phi1,phi2,s2,lambda_val)
    return beta2


def create_test_stat(df):
    beta2 = solve_beta(df)
    lambda_val = solve_lamb(df)
    
    phi2_calc = -3*beta2*lambda_val/2 + 3*lambda_val/2 + df['phi1'].mean()

    return  beta2#phi2_calc - df['phi2'].mean()



#print(solve_pi_seq_helper(23.125,15.625,19.375,.375,5))


passive_result1 = pd.read_csv('../GMM_est/fake_data/passive_data_0.csv',index_col=0)
active_result1 = pd.read_csv('../GMM_est/fake_data/active_data_0.csv',index_col=0)
seq_result1 = pd.read_csv('../GMM_est/fake_data/seq_data_0.csv',index_col=0)
for df in [passive_result1,active_result1,seq_result1]:
    print(solve_beta(df))
    print(create_test_stat(df))
    print('----')

1.0037419589907357
1.0037419589907357
----
0.9997799237883183
0.9997799237883183
----
0.6525845843579079
0.6525845843579079
----


In [13]:
def bootstrap_test(df,alpha=.05):
    
    test_stat =  create_test_stat(df)
    
    test_stats = []
    nobs = df.shape[0]
    for i in range(10):
        np.random.seed()
        bs_df = df.sample(frac=1,replace=True)
        bs_test_stat = create_test_stat(bs_df)
        test_stats.append(bs_test_stat)
        
    cv_upper = np.percentile(test_stats, 100*(1-alpha/2) , axis=0)
    cv_lower = np.percentile(test_stats, 100*(alpha/2) , axis=0)
    #print(test_stat,cv_lower,cv_upper)
    
    return  max(1*(1 <= cv_upper), 1*(0 >= cv_lower))


seq_result1 = pd.read_csv('../GMM_est/fake_data/seq_data_3.csv',index_col=0)
for df in [passive_result1,active_result1,seq_result1]:
    print(bootstrap_test(df))

1
1
0


In [14]:
# Create an empty DataFrame to store the results
results_list = []

for j in range(5):
    # Read the dataframes
    seq_result = pd.read_csv('../GMM_est/fake_data_test/seq_data_%s.csv' % j, index_col=0)
    active_result = pd.read_csv('../GMM_est/fake_data/active_data_%s.csv' % j, index_col=0)
    passive_result = pd.read_csv('../GMM_est/fake_data/passive_data_%s.csv' % j, index_col=0)

    # Initialize rejection count variables
    rejections_seq = 0
    rejections_active = 0
    rejections_passive = 0

    nsims = 25
    
    for i in range(nsims):
        # Resample from seq dataframe
        subsample_seq = seq_result.sample(frac=0.1)
        result_seq = bootstrap_test(subsample_seq)
        rejections_seq = rejections_seq+ result_seq
        
        # Resample from active dataframe
        subsample_active = active_result.sample(frac=0.1)
        result_active = bootstrap_test(subsample_active)
        rejections_active = rejections_active+ result_active

        # Resample from passive dataframe
        subsample_passive = passive_result.sample(frac=0.1)
        result_passive = bootstrap_test(subsample_passive)
        rejections_passive = rejections_passive+ result_passive
        #print('----%s,%s------'%(i,j))
        
    rejections_seq = rejections_seq/nsims
    rejections_active = rejections_active/nsims
    rejections_passive = rejections_passive/nsims
    
    
    # Add the results to the DataFrame
    results_list.append({'Round':j,'DataFrame': 'seq', 'Rejections': rejections_seq})
    results_list.append({'Round':j,'DataFrame': 'active', 'Rejections': rejections_active})
    results_list.append({'Round':j,'DataFrame': 'passive', 'Rejections': rejections_passive})

results_df = pd.DataFrame(data=results_list,columns=['Round','DataFrame','Rejections'])
        
print(results_df)  

    Round DataFrame  Rejections
0       0       seq        0.00
1       0    active        0.88
2       0   passive        0.96
3       1       seq        0.16
4       1    active        0.92
5       1   passive        0.92
6       2       seq        0.00
7       2    active        0.96
8       2   passive        1.00
9       3       seq        0.64
10      3    active        0.00
11      3   passive        0.00
12      4       seq        1.00
13      4    active        1.00
14      4   passive        1.00
